![](MSFTLogo.png)
# SQL Server Machine Learning Services
## 04 - Train and save a Python model using T-SQL

In this Notebook, you will train a machine learning model using the Python packages *scikit-learn* and *revoscalepy*. These Python libraries are already installed with SQL Server Machine Learning Services.

You load the modules and call the necessary functions to create and train the model using a SQL Server stored procedure. The model requires the data features you engineered in earlier lessons. Finally, you save the trained model to a SQL Server table.

## Split the sample data into training and testing sets

The following code creates a stored procedure called *PyTrainTestSplit* to divide the data in the *nyctaxi_sample table* into two parts: *nyctaxi_sample_training* and *nyctaxi_sample_testing*:


In [1]:
USE NYCTaxi;
GO

DROP PROCEDURE IF EXISTS PyTrainTestSplit;
GO

CREATE PROCEDURE [dbo].[PyTrainTestSplit] (@pct int)
AS

DROP TABLE IF EXISTS dbo.nyctaxi_sample_training
SELECT * into nyctaxi_sample_training FROM nyctaxi_sample WHERE (ABS(CAST(BINARY_CHECKSUM(medallion,hack_license)  as int)) % 100) < @pct

DROP TABLE IF EXISTS dbo.nyctaxi_sample_testing
SELECT * into nyctaxi_sample_testing FROM nyctaxi_sample
WHERE (ABS(CAST(BINARY_CHECKSUM(medallion,hack_license)  as int)) % 100) > @pct
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.040

To divide your data using a custom split, run the stored procedure, and type an integer that represents the percentage of data allocated to the training set. For example, the following statement would allocate 60% of data to the training set:

In [2]:
EXEC PyTrainTestSplit 60;
GO

(1025684 rows affected)

(660029 rows affected)

Total execution time: 00:00:24.992

After the data has been prepared, you can use it to train a model. You do this by calling a stored procedure that runs some Python code, taking as input the training data table. For this tutorial, you create two models, both binary classification models:

- The stored procedure *PyTrainScikit* creates a tip prediction model using the *scikit-learn* package
- The stored procedure *TrainTipPredictionModelRxPy* creates a tip prediction model using the *revoscalepy* package

Each stored procedure uses the input data you provide to create and train a logistic regression model. All Python code is wrapped in the system stored procedure, *sp_execute_external_script*. To make it easier to retrain the model on new data, you wrap the call to *sp_execute_external_script* in another stored procedure, and pass in the new training data as a parameter. This section will walk you through that process.

## PyTrainScikit

The following code creates the stored procedure *PyTrainScikit*. The stored procedure contains a definition of the input data, so you don't need to provide an input query:

In [3]:
DROP PROCEDURE IF EXISTS PyTrainScikit;
GO

CREATE PROCEDURE [dbo].[PyTrainScikit] (@trained_model varbinary(max) OUTPUT)
AS
BEGIN
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import numpy
import pickle
from sklearn.linear_model import LogisticRegression

##Create SciKit-Learn logistic regression model
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

SKLalgo = LogisticRegression()
logitObj = SKLalgo.fit(X, y)

##Serialize model
trained_model = pickle.dumps(logitObj)
',
@input_data_1 = N'
select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance, 
dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
from nyctaxi_sample_training
',
@input_data_1_name = N'InputDataSet',
@params = N'@trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT;
;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

In [4]:
-- Insert the trained model into table nyc_taxi_models:

DECLARE @model VARBINARY(MAX);
EXEC PyTrainScikit @model OUTPUT;
INSERT INTO nyc_taxi_models (name, model) VALUES('SciKit_model', @model);
GO


STDERR message(s) from external script: 
C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)

(1 row affected)

Total execution time: 00:00:35.857

In [5]:
-- Show the model - note that the code above could have specified more columns to serve as a model version:

SELECT * FROM nyc_taxi_models;
GO


(1 row affected)

Total execution time: 00:00:00.016

model,name
0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E6C6F6769737469630A4C6F67697374696352656772657373696F6E0A7100298171017D710228580700000070656E616C7479710358020000006C32710458040000006475616C7105895803000000746F6C7106473F1A36E2EB1C432D5801000000437107473FF0000000000000580D0000006669745F696E746572636570747108885811000000696E746572636570745F7363616C696E6771094B01580C000000636C6173735F776569676874710A4E580C00000072616E646F6D5F7374617465710B4E5806000000736F6C766572710C58040000007761726E710D58080000006D61785F69746572710E4B64580B0000006D756C74695F636C617373710F680D5807000000766572626F736571104B00580A0000007761726D5F737461727471118958060000006E5F6A6F627371124E5808000000636C61737365735F7113636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A7114636E756D70790A6E6461727261790A71154B008571164301627117877118527119284B014B0285711A636E756D70790A64747970650A711B58020000006934711C4B004B0187711D52711E284B0358010000003C711F4E4E4E4AFFFFFFFF4AFFFFFFFF4B007471206289430800000000010000007121747122625805000000636F65665F7123681468154B008571246817877125527126284B014B014B04867127681B5802000000663871284B004B0187712952712A284B03681F4E4E4E4AFFFFFFFF4AFFFFFFFF4B0074712B62894320865D0CAABF73A0BF70BF5DA718A7A53F12D0377F8CB682BE70C34C983BCC123F712C74712D62580A000000696E746572636570745F712E681468154B0085712F6817877130527131284B014B01857132682A894308CB2259C25B0DA03F71337471346258070000006E5F697465725F7135681468154B008571366817877137527138284B014B01857139681B58020000006934713A4B004B0187713B52713C284B03681F4E4E4E4AFFFFFFFF4AFFFFFFFF4B0074713D6289430419000000713E74713F6258100000005F736B6C6561726E5F76657273696F6E71405806000000302E32302E32714175622E,SciKit_model


## TrainTipPredictionModelRxPy

The next stored procedure uses the *revoscalepy* package, which is a new package for Python. It contains objects, transformation, and algorithms similar to those provided for the R language's RevoScaleR package.

By using *revoscalepy*, you can create remote compute contexts, move data between compute contexts, transform data, and train predictive models using popular algorithms such as logistic and linear regression, decision trees, and more. For more information, see revoscalepy module in SQL Server and revoscalepy function reference.

The following statement creates the stored procedure *TrainTipPredictionModelRxPy*. Because the stored procedure already includes a definition of the input data, you don't need to provide an input query:

In [6]:
DROP PROCEDURE IF EXISTS TrainTipPredictionModelRxPy;
GO

CREATE PROCEDURE [dbo].[TrainTipPredictionModelRxPy] (@trained_model varbinary(max) OUTPUT)
AS
BEGIN
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import numpy
import pickle
from revoscalepy.functions.RxLogit import rx_logit

## Create a logistic regression model using rx_logit function from revoscalepy package
logitObj = rx_logit("tipped ~ passenger_count + trip_distance + trip_time_in_secs + direct_distance", data = InputDataSet);

## Serialize model
trained_model = pickle.dumps(logitObj)
',
@input_data_1 = N'
select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance, 
dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
from nyctaxi_sample_training
',
@input_data_1_name = N'InputDataSet',
@params = N'@trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT;
;
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

This stored procedure performs the following steps as part of model training:

The **SELECT** query applies the custom scalar function *fnCalculateDistance* to calculate the direct distance between the pick-up and drop-off locations. The results of the query are stored in the default Python input variable, *InputDataset*.

The binary variable *tipped* is used as the label or outcome column, and the model is fit using these feature columns: *passenger_count, trip_distance, trip_time_in_secs*, and *direct_distance*.

The trained model is serialized and stored in the Python variable *logitObj*. By adding the T-SQL keyword **OUTPUT**, you can add the variable as an output of the stored procedure. In the next step, that variable is used to insert the binary code of the model into a database table *nyc_taxi_models*. This mechanism makes it easy to store and re-use models.

This code inserts the trained *revoscalepy* model into the table *nyc_taxi_models*, and check the contents - you should have two models now:

In [7]:
DECLARE @model VARBINARY(MAX);
EXEC TrainTipPredictionModelRxPy @model OUTPUT;
INSERT INTO nyc_taxi_models (name, model) VALUES('revoscalepy_model', @model);
GO

-- Check the models
SELECT * FROM nyc_taxi_models;
GO

STDOUT message(s) from external script: 
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.089 seconds 

Starting values (iteration 1) time: 0.304 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.178 seconds 

Iteration 2 time: 0.179 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.174 seconds 

Iteration 3 time: 0.176 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.173 seconds 

Iteration 4 time: 0.176 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.175 seconds 

Iteration 5 time: 0.176 secs.
Rows Read: 1025684, Total Rows Processed: 1025684
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation.
, Total Chunk Time: 0.175 seconds 

Iteration 6 time: 0.177 secs.
Rows Read: 1025684, Total Rows Processed: 1025684
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation.

STDOUT message(s) from external script: 
, Total Chunk Time: 0.179 seconds 

Iteration 7 time: 0.181 secs.
Rows Read: 1025684, Total Rows Processed: 1025684
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation.
, Total Chunk Time: 0.177 seconds 

Iteration 8 time: 0.178 secs.
Rows Read: 1025684, Total Rows Processed: 1025684
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation.
, Total Chunk Time: 0.175 seconds 

Iteration 9 time: 0.177 secs.

Elapsed computation time: 1.760 secs.

(1 row affected)

(2 rows affected)

Total execution time: 00:00:18.164

model name 0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E6C6F6769737469630A4C6F67697374696352656772657373696F6E0A7100298171017D710228580700000070656E616C7479710358020000006C32710458040000006475616C7105895803000000746F6C7106473F1A36E2EB1C432D5801000000437107473FF0000000000000580D0000006669745F696E746572636570747108885811000000696E746572636570745F7363616C696E6771094B01580C000000636C6173735F776569676874710A4E580C00000072616E646F6D5F7374617465710B4E5806000000736F6C766572710C58040000007761726E710D58080000006D61785F69746572710E4B64580B0000006D756C74695F636C617373710F680D5807000000766572626F736571104B00580A0000007761726D5F737461727471118958060000006E5F6A6F627371124E5808000000636C61737365735F7113636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A7114636E756D70790A6E6461727261790A71154B008571164301627117877118527119284B014B0285711A636E756D70790A64747970650A711B58020000006934711C4B004B0187711D52711E284B0358010000003C711F4E4E4E4AFFFFFFFF4AFFFFFFFF4B007471206289430800000000010000007121747122625805000000636F65665F7123681468154B008571246817877125527126284B014B014B04867127681B5802000000663871284B004B0187712952712A284B03681F4E4E4E4AFFFFFFFF4AFFFFFFFF4B0074712B62894320865D0CAABF73A0BF70BF5DA718A7A53F12D0377F8CB682BE70C34C983BCC123F712C74712D62580A000000696E746572636570745F712E681468154B0085712F6817877130527131284B014B01857132682A894308CB2259C25B0DA03F71337471346258070000006E5F697465725F7135681468154B008571366817877137527138284B014B01857139681B58020000006934713A4B004B0187713B52713C284B03681F4E4E4E4AFFFFFFFF4AFFFFFFFF4B0074713D6289430419000000713E74713F6258100000005F736B6C6561726E5F76657273696F6E71405806000000302E32302E32714175622E SciKit_model 0x8003637265766F7363616C6570792E66756E6374696F6E732E52784C6F6769740A52784C6F676974526573756C74730A7100298171017D710228580B0000005F6D6F64656C5F747970657103580700000072784C6F676974710458080000005F666F726D756C617105584E000000746970706564207E2070617373656E6765725F636F756E74202B20747269705F64697374616E6365202B20747269705F74696D655F696E5F73656373202B206469726563745F64697374616E6365710658070000005F706172616D7371077D710828580E00000072656D6F76654D697373696E67737109885806000000766172546F6C710A473D719799812DEA11580800000072436F6E64546F6C710B473D06849B86A12B9B5807000000466F726D756C61710C6806580400000043756265710D89580F0000004375626550726564696374696F6E73710E89580C000000726F7753656C656374696F6E710F4E580A0000007472616E73666F726D7371104E58100000007472616E73666F726D4F626A6563747371114E580D0000007472616E73666F726D46756E6371124E580D0000007472616E73666F726D5661727371134E58110000007472616E73666F726D5061636B6167657371144E580900000064726F7046697273747115895807000000636F76436F65667116895807000000636F7644617461711789580E000000696E697469616C5F76616C75657371184E580800000064726F704D61696E711988580D000000626C6F636B7350657252656164711A4B01580D0000004D6178497465726174696F6E73711B4B19581500000062557365725365744D6178497465726174696F6E73711C89580E000000436F656666546F6C6572616E6365711D473EB0C6F7A0B5ED8D581A0000004F626A65637469766546756E6374696F6E546F6C6572616E6365711E473E45798EE2308C3A58110000004772616469656E74546F6C6572616E6365711F473EB0C6F7A0B5ED8D580D0000002E2E7278446174614672616D6571204E5806000000466164644350712163636F6C6C656374696F6E730A4F726465726564446963740A712229527123580F00000062436F6D707574654C6F7748696768712488735808000000726561645661727371255D71262858060000007469707065647127580B000000666172655F616D6F756E747128580F00000070617373656E6765725F636F756E7471295811000000747269705F74696D655F696E5F73656373712A580D000000747269705F64697374616E6365712B580F0000006469726563745F64697374616E6365712C655807000000566572626F7365712D4B0058050000005072696E74712E89580E0000005265706F727450726F6772657373712F88580C0000005265706F727454696D696E6771304B01580E00000053686F77497465726174696F6E73713189581600000053686F7744657461696C6564497465726174696F6E73713289580400000074656D707133585A000000433A5C5C50726F6772616D446174615C5C4D5353514C5345525645525C5C54656D702D50595C5C417070636F6E7461696E6572315C5C34434143344441412D453342342D343846462

Now proceed to the **05 - Run Batch and Single-score Predictions using T-SQL** Jupyter Notebook.